Edward Daou (ID:21400357)

In [ ]:
import glob
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from skimage.measure import find_contours
from scipy.spatial.distance import dice
from skimage.transform import rescale
from scipy.io import loadmat
import cv2

from scipy import ndimage as ndi
from skimage import io as skio
from skimage.filters import threshold_otsu
import skimage.morphology as morpho  
from skimage.segmentation import watershed 
from skimage.draw import line
from skimage.transform import rescale
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage import img_as_float
from skimage.segmentation import chan_vese
from skimage.segmentation import checkerboard_level_set
from skimage.segmentation import disk_level_set


The aim of this practical work is to experiment various segmentation methods on medical images: CT for kidney segmentation, brain MRI for corpus callosum segmentation, temporal MRI sequences for heart segmentation (in particular left ventricle), dermoscopic images of skin lesions.
For each method, you have to test and evaluate it on as many images as possible.


In [ ]:
if 'google.colab' in str(get_ipython()):
  from google_drive_downloader import GoogleDriveDownloader as gdd

  gdd.download_file_from_google_drive(file_id='1E-F5x83qaPRUrcdusfyOSgiZWf0pLDFl',
                                      dest_path='./abdominalCT.zip',
                                      unzip=True)

  gdd.download_file_from_google_drive(file_id='1UTSW2nGWV8SBE_ILZQkM-jQvnfoQKDlm',
                                      dest_path='./brainMRI.zip',
                                      unzip=True)  

  gdd.download_file_from_google_drive(file_id='1BZtShVk7LVG032GOlcLs0K4Eaj6gnBed',
                                      dest_path='./MRIheart.zip',
                                      unzip=True)    

  gdd.download_file_from_google_drive(file_id='1BoU4S9xFgKaCtquU03kZQbH3Yv42oU_-',
                                      dest_path='./skinlesion.zip',
                                      unzip=True)   
else:
  print('You are not using Colab. Please define working_dir with the absolute path to the folder where you downloaded the data')

# Please modify working_dir only if you are using your Anaconda (and not Google Colab)
# You should write the absolute path of your working directory with the data
Working_directory="data/"                              

## Abdominal CT 

You have at your disposal 6 abdominal CT scans of different subjects. Subjects may have renal tumor. You also have the manual segmentations for both kidney and, when present, the tumor.

In [ ]:
print(Working_directory)

In [ ]:
abdominalCT_path = Working_directory + 'abdominalCT'
print(abdominalCT_path)
listImagesabdCT=glob.glob(abdominalCT_path + '/*-seg.tiff')
print('There are', len(listImagesabdCT),  'abdomical CT images')
print(listImagesabdCT)

In [ ]:
# Choose a figure and plot it with the ground truth segmentation
indexIm=0 # between 0 and 5

# Abdominal CT
filename_Segmentation = listImagesabdCT[indexIm]
im_Seg = imread(filename_Segmentation)
filename = filename_Segmentation[:-9] + '.tiff'
imG = imread(filename) 


print('Reading image ', filename)
print(np.unique(im_Seg))

if imG.shape != im_Seg.shape:
  raise NameError('image and mask should have the same shape, problem...')  

# In Im Seg, we may have two values: 127 is for kidney and 255 for renal tumor
maskKidney=im_Seg==127
if np.sum(maskKidney)==0:
  print('There is no kidney')
contourMaskKidney = find_contours(maskKidney, 0.5)

maskTumor=im_Seg==255
if np.sum(maskTumor)==0:
  print('There is no tumor')
contourMaskTumor = find_contours(maskTumor, 0.5)

fig = plt.figure(figsize=(17, 7))
grid = AxesGrid(fig, 111,
                nrows_ncols = (1, 5),
                axes_pad = 0.5)
grid[0].imshow(imG, cmap='gray')
grid[0].axis('off')
grid[0].set_title("Original image")
grid[1].imshow(maskKidney,cmap='gray')
grid[1].axis('off')
grid[1].set_title("Segmentation Mask Kidney")
grid[2].imshow(maskTumor,cmap='gray')
grid[2].axis('off')
grid[2].set_title("Segmentation Mask Tumor")
grid[3].imshow(imG, cmap='gray')
for contour in contourMaskKidney:
  grid[3].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
grid[3].axis('off')
grid[3].set_title("Image with mask kidney")
grid[4].imshow(imG, cmap='gray')
for contour in contourMaskTumor:
  grid[4].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
grid[4].axis('off')
grid[4].set_title("Image with mask tumor")


# Brain MRI

Here you can select medial slices of the brain of 4 different subjects. You also have manual segmentations of the corpus callosum.

In [ ]:
brainMRI_path = Working_directory + 'brainMRI'

listImagesbrainMRI=glob.glob(brainMRI_path + '/*-seg.png')
print('There are', len(listImagesbrainMRI),  'brain MRI images')
print(listImagesbrainMRI)

In [ ]:
# Choose a figure and plot it with the ground truth segmentation
indexIm=3 # between 0 and 3

# Brain MRI
filename_Segmentation = listImagesbrainMRI[indexIm]
print(filename_Segmentation)
im_Seg = imread(filename_Segmentation)
filename = filename_Segmentation[:-8] + '.png'
imG = imread(filename) 

print('Reading image ', filename)

if imG.shape != im_Seg.shape:
  raise NameError('image and mask should have the same shape, problem...')  

# In Im Seg we have masks of the corpus callosum
maskCC=im_Seg==255
contourMask = find_contours(maskCC, 0.5)

fig = plt.figure(figsize=(17, 7))
grid = AxesGrid(fig, 111,
                nrows_ncols = (1, 3),
                axes_pad = 0.5)
grid[0].imshow(imG, cmap='gray')
grid[0].axis('off')
grid[0].set_title("Original image")
grid[1].imshow(maskCC,cmap='gray')
grid[1].axis('off')
grid[1].set_title("Segmentation Mask\n Corpus Callosum")
grid[2].imshow(imG, cmap='gray')
for contour in contourMask:
  grid[2].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
grid[2].axis('off')
grid[2].set_title("Image with segmentation\n corpus callosum")


# Skin lesions

In this section, you can try to segment skin lesions (both nevi and melanoma).  You also have manual segmentations. Be careful to the hair...

In [ ]:
skinlesion_path = Working_directory + 'skinlesion'

listImages=glob.glob(skinlesion_path + '/*.jpg')
N=len(listImages)
print('There are {} skin lesion images'.format(N))
print(listImages)


In [ ]:
# Let's load the images, rescale them so that the computations are faster and plot them
# Choose the index of the skin lesion image you want to analyze
indexIm=10
####

filename = listImages[indexIm]
im = imread(filename)
im=im[2:-2,2:-2,:] # remove border (it contains artifacts)
# To simplify things, let's choose only one channel (gray image)
# Select a channel (0 for Red, 1 for Green and 2 for Blue)
channel=2
##
imG = im[:,:,channel] 
imG = rescale(imG, 0.25,anti_aliasing=True)


filename_Segmentation = filename[:-4] + '_Segmentation.png'
im_Seg = imread(filename_Segmentation) # Value 0 or 255
im_Seg=im_Seg[2:-2,2:-2] # remove border (it contains artifacts)
im_Seg = rescale(im_Seg, 0.25, anti_aliasing=False, order=0,  preserve_range=True)
maskSL = im_Seg==255 
im_Seg_expand = np.expand_dims(maskSL, axis=2)

fig = plt.figure(figsize=(17, 7))
grid = AxesGrid(fig, 111,
                nrows_ncols = (1, 2),
                axes_pad = 0.5)
grid[0].imshow(imG,cmap='gray')
grid[0].axis('off')
grid[0].set_title("Chosen channel from original image")
grid[1].imshow(maskSL,cmap='gray')
grid[1].axis('off')
grid[1].set_title("Mask")
#grid[2].imshow(im_Seg_expand*imG)
#grid[2].axis('off')
#grid[2].set_title("Image with mask")

# MRI heart

The last section is about MRI sequences of the heart. Your goal is to segment the left ventricule. Be careful, the segmentation is not a maks but a series of points (landmarks). To obtain a binary mask, you should first interpolate the points (using for instance a spline).


In [ ]:
MRIheart_path = Working_directory + 'MRIheart/'


In [ ]:
data=loadmat(MRIheart_path + 'dataMRIheart.mat')
data=data['data']
seg=loadmat(MRIheart_path + 'segMRIheart.mat')
seg=seg['seg']

print('MRI volume of the heart composed of', data.shape[2], 'slices along the z axis and', data.shape[3], 
'temporal frames. Each slice is an image ', data.shape[0], ' x ',  data.shape[1])
print('For each slice and at each time frame we have a manual segmentation composed of',seg[4,4].shape[0] , '2D landmarks')

print('Be careful, some slices do not have the left ventricle and the manual segmentation is not simply empty but it contains the value:', seg[0,0] )

In [ ]:
plt.figure(figsize=(20, 9))
plt.suptitle('Different slices at the same time frame with the red manual segmentation at the bottom')
plt.subplot(2, 5, 1)
plt.imshow(data[:,:,4,1],cmap="gray")
plt.gca().invert_yaxis() 

plt.subplot(2, 5, 2)
plt.imshow(data[:,:,5,1],cmap="gray")
plt.gca().invert_yaxis() 

plt.subplot(2, 5, 3)
plt.imshow(data[:,:,6,1],cmap="gray")
plt.gca().invert_yaxis() 

plt.subplot(2, 5, 4)
plt.imshow(data[:,:,7,1],cmap="gray")
plt.gca().invert_yaxis() 

plt.subplot(2, 5, 5)
plt.imshow(data[:,:,8,1],cmap="gray")
plt.gca().invert_yaxis() 

plt.subplot(2, 5, 6)
plt.imshow(data[:,:,4,1],cmap="gray")
plt.gca().invert_yaxis() 
plt.scatter(seg[4,1][:,0], seg[4,1][:,1], c='r',alpha=0.1) 

plt.subplot(2, 5, 7)
plt.imshow(data[:,:,5,1],cmap="gray")
plt.gca().invert_yaxis() 
plt.scatter(seg[5,1][:,0], seg[5,1][:,1], c='r',alpha=0.1) 

plt.subplot(2, 5, 8)
plt.imshow(data[:,:,6,1],cmap="gray")
plt.gca().invert_yaxis() 
plt.scatter(seg[6,1][:,0], seg[6,1][:,1], c='r',alpha=0.1) 

plt.subplot(2, 5, 9)
plt.imshow(data[:,:,7,1],cmap="gray")
plt.gca().invert_yaxis() 
plt.scatter(seg[7,1][:,0], seg[7,1][:,1], c='r',alpha=0.1) 

plt.subplot(2, 5, 10)
plt.imshow(data[:,:,8,1],cmap="gray")
plt.gca().invert_yaxis() 
plt.scatter(seg[8,1][:,0], seg[8,1][:,1], c='r',alpha=0.1) 

Useful functions

In [ ]:
def strel(forme,taille,angle=45):
    """renvoie un element structurant de forme  
     'diamond'  boule de la norme 1 fermee de rayon taille
     'disk'     boule de la norme 2 fermee de rayon taille
     'square'   carre de cote taille (il vaut mieux utiliser taille=impair)
     'line'     segment de langueur taille et d'orientation angle (entre 0 et 180 en degres)
      (Cette fonction n'est pas standard dans python)
    """

    if forme == 'diamond':
        return morpho.diamond(taille)
    if forme == 'disk':
        return morpho.disk(taille)
    if forme == 'square':
        return morpho.square(taille)
    if forme == 'line':
        angle=int(-np.round(angle))
        angle=angle%180
        angle=np.float32(angle)/180.0*np.pi
        x=int(np.round(np.cos(angle)*taille))
        y=int(np.round(np.sin(angle)*taille))
        if x**2+y**2 == 0:
            if abs(np.cos(angle))>abs(np.sin(angle)):
                x=int(np.sign(np.cos(angle)))
                y=0
            else:
                y=int(np.sign(np.sin(angle)))
                x=0
        rr,cc=line(0,0,y,x)
        rr=rr-rr.min()
        cc=cc-cc.min()
        img=np.zeros((rr.max()+1,cc.max()+1) )
        img[rr,cc]=1
        return img
    raise RuntimeError('Erreur dans fonction strel: forme incomprise')



Evaluation of results: quantitative (when a reference segmentation is available) and qualitative (visual) - To be done for each obtained result!


In [ ]:
# Quantitative evaluation using Dice index (you can also use the dice function of scipy)
def Dice(segm,segm_ref):
    if segm.shape != segm_ref.shape:
        raise ValueError("Les deux images doivent avoir la meme taille.")
    segm = np.asarray(segm).astype(np.bool)
    segm_ref = np.asarray(segm_ref).astype(np.bool)
    intersection = np.logical_and(segm, segm_ref)
    dice = 2.0 * intersection.sum() / (segm.sum()+segm_ref.sum())
    return dice

# Visual comparison (segm can be a segmentation mask or its contour)
# you can also use the find_contours function
def ComparVisu(image,segm):
    return np.maximum(image,segm)
def Contour(segm):
    se=morpho.disk(1)
    contour=morpho.dilation(segm,se)-segm
    # or dilation - erosion to get a 2 pixels width contour (symmetric and more visible)
    return contour


1. Segmentation using thresholding
- compare the histograms of images of the same modality. Are the conclusions the same on CT images and on MRI images?
- consequences on manual thresholding?
- test the automatic Otsu's thresholding method. Conclusions?
- test the automatic k-means method (2 classes or more). Conclusions?


Notes: 
- we select 4 images for each modality
- we choose to focus mainly on the left ventricule for the heart mri (so we leave out the myocardium manual annotations)

In [ ]:
# Histogram
from scipy.interpolate import splprep, splev #library for spline interpolation
from skimage.draw import polygon #used to draw polygon with a set of points



#skin lesions histogtam
plt.figure(figsize=(20, 9))

#arrays to store the original images and their respective manual annotations
skinImages=[]
manualSkin=[]

#iterating over each image we want to choose and showing each respective histogram
for i in range (0,4):
    filename = listImages[i]
    im = imread(filename)
    im=im[2:-2,2:-2,:] 

    channel=2
    imG = im[:,:,channel] 
    imG = rescale(imG, 0.25,anti_aliasing=True)
    im=imG


    plt.suptitle('dermoscopic histograms (skin lesions)')
    plt.subplot(4, 4, i+1)
    ax = plt.hist(im.ravel(), bins = 256)
    #saving the image for future use
    skinImages.append(im)
    
    #saving the manual mask for future use
    filename_Segmentation = filename[:-4] + '_Segmentation.png'
    im_Seg = imread(filename_Segmentation) # Value 0 or 255
    im_Seg=im_Seg[2:-2,2:-2] # remove border (it contains artifacts)
    im_Seg = rescale(im_Seg, 0.25, anti_aliasing=False, order=0,  preserve_range=True)
    maskSL = im_Seg==255 
    manualSkin.append(maskSL)
    
plt.show()


#abdominalCT
plt.figure(figsize=(20, 9))

#arrays to store the original images and their respective manual annotations (one for the kidneys, one for tumor)
abdoImages=[]
manualAbdoKidney=[]
manualAbdoTumor=[]

for i in range (0,4):
    filename_Segmentation = listImagesabdCT[i]
    im_Seg = imread(filename_Segmentation)
    filename = filename_Segmentation[:-9] + '.tiff'
    imGCT = imread(filename) 
    imCT=imGCT

    plt.suptitle('CT histograms (abdominal)')
    plt.subplot(4, 4, i+1)
    ax = plt.hist(imCT.ravel(), bins = 256)
    #saving the image for future use
    abdoImages.append(imCT)
    
    #saving the manual masks for future use
    maskKidney=im_Seg==127
    if np.sum(maskKidney)==0:
          print(f'There is no kidney  (index={i})')

    maskTumor=im_Seg==255
    if np.sum(maskTumor)==0:
          print(f'There is no tumor (index={i})') 
            
    manualAbdoKidney.append(maskKidney)
    manualAbdoTumor.append(maskTumor)

plt.show()

#brainMRI
plt.figure(figsize=(20, 9))

#arrays to store the original images and their respective manual annotations
brainImages=[]
manualBrain=[]

for i in range (0,4):
    filename_Segmentation = listImagesbrainMRI[i]
    im_Seg = imread(filename_Segmentation)
    filename = filename_Segmentation[:-8] + '.png'
    imGBrain = imread(filename) 
    imBrain=imGBrain


    plt.suptitle('MRI histograms (brain)')
    plt.subplot(5, 5, i+1)
    ax = plt.hist(imBrain.ravel(), bins = 256)
    #saving the image for future use
    brainImages.append(imBrain)
    #saving the manual mask for future use
    maskCC=im_Seg==255
    manualBrain.append(maskCC)
    



#heartMRI
plt.figure(figsize=(20, 9))

#arrays to store the original images and their respective manual annotations
heartImages=[]
manualHeart=[]

for i in range (0,4):
    imHeart=data[:,:,3,i]
    plt.suptitle('MRI histograms (heart)')
    plt.subplot(4, 4, i+1)
    ax = plt.hist(imHeart.ravel(), bins = 256)
    #saving the image for future use
    heartImages.append(imHeart)
    
    #saving the manual mask for future use
    maskPts=seg[3,i]    
    
    #extracting all points until point (0,0) (used to separate the left ventricule and myocaridum manual annotations)
    #our study is mainly interested in the left ventricule segmentation
    zeroIndex=np.where((maskPts[:,0]==0)&(maskPts[:,1]==0))[0][0]
    maskPts=maskPts[0:zeroIndex]
    
    #we smooth the manual mask with an spline interpolation of 100 points
    splineInfo,numPts=splprep([maskPts[:,1],maskPts[:,0]])
    morePts=np.linspace(0, 1, 100)
    ptsUpdated=splev(morePts, splineInfo)
    mask=np.zeros_like(imHeart)
    poly=polygon(ptsUpdated[0], ptsUpdated[1], mask.shape)
    mask[poly[0],poly[1]]=1
    manualHeart.append(mask)
    
    
plt.show()



CONCLUSIONS CONCERNING THE HISTOGRAMS:
- For the MRIs and CT images, most of the intensity values are 0 (a very high peak in that value) which means that the tissues/structures we want to study occupy a small portion of these images compared to the background. Skin occupy the whole area of the image in the dermoscopic images
- The CT and MRIs differ by one main factor: the CT histogram appears discrete while the MRI histogram show more continuous values. This means that CT images are less noisy but show more artifacts than MRI images (each few peaks of the CT scan would correspond to a specific type of bone/tissue)
- The dermoscopic images, similarly to MRIs, show a less discrete graph. This indicates potential noise and less artifacts.

Notes:
- the red lines in the following graphs (for this entire project) correspond to the manual delineations 
- for each abdominal ct image, the blue line corresponds to the tumor (if any) and the red ones to the kidneys
- we use the dice distance measure (the lower it is, the better the segmentation)
- segmentations are shown on top of the original image (white pixels corresponds to the segmentation)
- the brain MRI distributions are less consistent than the heart MRI ones (since we have a same heart at different temporal stamps for the heart MRI images)

In [ ]:
# manual thresholding

#4 images for each modality are thresholded using the following thresholding ratios (1e-59, 0.2, 0.5 and 0.9)
def thresh(images, masks, part, title):
    for i in range (0,4):
        image=images[i]
        #takes in account that we have 2 mmanual masks for the abdominal cts
        if(part=='abdo'):
            maskK=masks[0][i]
            maskT=masks[1][i]
            maskK_c= find_contours(masks[0][i], 0.5)
            maskT_c= find_contours(masks[1][i], 0.5)
        else:
            mask=masks[i]
            contourMask = find_contours(mask, 0.5) #extract boundaries of the mask
            
        # we need to extract dark lesions for the skin images, therefore we need to keep the lower intensity values
        if (part=='skin'):
            binary1 = (image < 1e-50*np.max(image))
            binary2 = (image < .2*np.max(image))
            binary3 = (image < .5*np.max(image))
            binary4 = (image < .9*np.max(image))
            
        #higher intensity values kept for other modalities
        else:
            binary1 = (image > 1e-50*np.max(image))
            binary2 = (image > .2*np.max(image))
            binary3 = (image > .5*np.max(image))
            binary4 = (image > .9*np.max(image))
        fig = plt.figure(figsize=(17, 7))
        plt.figure(figsize=(20, 9))
        if(i==0):
            plt.suptitle(title)
        
        if part=='abdo':
            #show the original image
            plt.subplot(2,5,1)
            plt.imshow(image, cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
                
            #show the thresholded image with a threshold value of 0.2
            plt.subplot(2,5,2)
            plt.title(f"dice(k) ={round(dice(binary1.ravel(),maskK.ravel()),3)}, dice(t) ={round(dice(binary1.ravel(),maskT.ravel()),3)}")
            plt.imshow(ComparVisu(image,binary1*np.max(image)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
                
            #show the thresholded image with a threshold value of 0.4
            plt.subplot(2,5,3)
            plt.title(f"dice(k) ={round(dice(binary2.ravel(),maskK.ravel()),3)}, dice(t) ={round(dice(binary2.ravel(),maskT.ravel()),3)}")
            plt.imshow(ComparVisu(image,binary2*np.max(image)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
                
            #show the thresholded image with a threshold value of 0.5
            plt.subplot(2,5,4)
            plt.title(f"dice(k) ={round(dice(binary3.ravel(),maskK.ravel()),3)}, dice(t) ={round(dice(binary3.ravel(),maskT.ravel()),3)}")
            plt.imshow(ComparVisu(image,binary3*np.max(image)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
                
            #show the thresholded image with a threshold value of 0.9
            plt.subplot(2,5,5)
            plt.title(f"dice(k) ={round(dice(binary4.ravel(),maskK.ravel()),3)}, dice(t) ={round(dice(binary4.ravel(),maskT.ravel()),3)}")
            plt.imshow(ComparVisu(image,binary4*np.max(image)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
            plt.show()
            
        else:
            #show the original image
            plt.subplot(2,5,1)
            plt.imshow(image, cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
                
            #show the thresholded image with a threshold value of 0.2
            plt.subplot(2,5,2)
            plt.title(f"dice distance ={dice(binary1.ravel(),mask.ravel())}")
            plt.imshow(ComparVisu(image,binary1*np.max(image)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            
            #show the thresholded image with a threshold value of 0.4
            plt.subplot(2,5,3)
            plt.title(f"dice distance ={dice(binary2.ravel(),mask.ravel())}")
            plt.imshow(ComparVisu(image,binary2*np.max(image)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            
            #show the thresholded image with a threshold value of 0.5
            plt.subplot(2,5,4)
            plt.title(f"dice distance ={dice(binary3.ravel(),mask.ravel())}")
            plt.imshow(ComparVisu(image,binary3*np.max(image)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
                
            #show the thresholded image with a threshold value of 0.9
            plt.subplot(2,5,5)
            plt.title(f"dice distance ={dice(binary4.ravel(),mask.ravel())}")
            plt.imshow(ComparVisu(image,binary4*np.max(image)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.show()

thresh(skinImages, manualSkin, 'skin','basic threshold for skin lesions (thresh = 1e-50, 0.2, 0.5 and 0.9 left to right)')
thresh(abdoImages, [manualAbdoKidney,manualAbdoTumor],'abdo','basic threshold for abdominal CT (thresh = 1e-50, 0.2, 0.5 and 0.9 left to right)')
thresh(brainImages,manualBrain,'brain', 'basic threshold for brain mri (thresh = 1e-50, 0.2, 0.5 and 0.9 left to right)')
thresh(heartImages,manualHeart, 'heart','basic threshold for heart mri (thresh = 1e-50, 0.2, 0.5 and 0.9 left to right)')


CONCLUSIONS CONCERNING THE BASIC THRESHOLDING OF THE MODALITIES:
- For CT images, since the histograms are discrete, a slight change in the threshold value will not influence the result much (as can be seen for all ct images between the 0.2 and 0.5 values threshold). Thresholding is more unstable for MRIs and dermoscopic images.
- For CT and MRI images, the majority of pixels are dark because most of the image corresponds to background or low-density tissues. High intensity values appear mainly near the edges of the body, corresponding to denser structures. There is little to no contrast inside between soft tissues, making the thresholding more challenging.
- Identifying a general threshold value for the skin lesions is challenging since average intensity values in these regions vary much from an image to another (as can been seen from the color intensity of the lesions in the original images)

In [ ]:

# Otsu thresholding

#otsu thresholding on 4 images for each modality 
def OtsuThresh(images,  masks, part, title):
    for i in range (0,4):
        image=images[i]
        threshOtsu = threshold_otsu(image)
        if(part=='abdo'):
            maskK=masks[0][i]
            maskT=masks[1][i]
            maskK_c= find_contours(masks[0][i], 0.5)
            maskT_c= find_contours(masks[1][i], 0.5)
        else:
            mask=masks[i]
            contourMask = find_contours(mask, 0.5)
        if (part=='skin'):
            binary1 = (image < threshOtsu)
        else:
            binary1 = (image > threshOtsu)
        fig = plt.figure(figsize=(17, 7))
        plt.figure(figsize=(20, 9))
        if(i==0):
            plt.suptitle(title)
        
        if part=='abdo':
            plt.subplot(2,2,1)
            plt.imshow(image, cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
            plt.subplot(2,2,2)
            plt.title(f"dice(k) ={round(dice(binary1.ravel(),maskK.ravel()),3)}, dice(t) ={round(dice(binary1.ravel(),maskT.ravel()),3)}")
            plt.imshow(ComparVisu(image,binary1*np.max(image)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
            plt.show()
            
        else:
            plt.subplot(2,2,1)
            plt.imshow(image, cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.subplot(2,2,2)
            plt.title(f"dice distance ={dice(binary1.ravel(),mask.ravel())}")
            plt.imshow(ComparVisu(image,binary1*np.max(image)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.show()

OtsuThresh(skinImages, manualSkin,'skin',  'Otsu thresholding for skin lesions')
OtsuThresh(abdoImages, [manualAbdoKidney,manualAbdoTumor], 'abdo','Otsu thresholding for abdominal CT')
OtsuThresh(brainImages,manualBrain,'brain', 'Otsu thresholding for brain mri')
OtsuThresh(heartImages, manualHeart,'heart','Otsu thresholding for heart mri')

CONCLUSIONS CONCERNING OTSU THRESHOLDING:
Otsu threshold aims at separating background from foreground information while minimizing intra class variance

-Otsu thresholding produced very different results on the brain mri images. While the edges of the overall body always appear on the thresholded image, the low-noise tissues insides either appear mostly or do not.
- Because of the very high peak at 0, otsu's thresholding keeps everything except the background for abdominal ct, making it not a reliable choice.
-because of the high contrast between the skin lesion and the rest of the skin. Otsu thresholding identifies that region well. However, the hairs are also contrasting with the background. Otsu's thresholding therefore capture the hair too which need to be removed.

In [ ]:
#custom thresholding method for skin images based on otsu's thresholding
from skimage.measure import label, regionprops
from skimage.morphology import opening, disk
from scipy.ndimage import binary_fill_holes

#helper method to find the most 'circular' region corresponding to the lesion
# we pass minAreaRatio so that the area is not too small (corresponding to noise) 
#and maxEccentricity to not take in consideration connected components with high eccentricity
def findCircular(image, minAreaRatio=1/20, eccentricityMax=0.9):
    labels=label(image)
    best=-1000 #variable for best circularity
    choseLabel=None #the label to keep

    #iterate over regions
    for region in regionprops(labels):
        if region.area<minAreaRatio*(len(image)*len(image[0])):
            continue
        perimeter=region.perimeter
        #calculate circularity and eccentricity
        circularity=(4*np.pi*region.area)/(perimeter*perimeter)
        eccentricity=region.eccentricity

        #skip if eccentricity is too high
        if eccentricity>eccentricityMax:
            continue
        #save the region with best circularity
        if circularity>best:
            best=circularity
            choseLabel=region.label
    result=np.zeros_like(image)
    result[labels==choseLabel]=1
    return result

resultSkinOtsu=[] #save the results for future use

#function to threshold the skin images
def OtsuThreshSkin(images,  masks, part, title):
    for i in range (0,4):
        #step 1: otsu thresholding
        image=images[i]
        threshOtsu = threshold_otsu(image)
        mask=masks[i]
        contourMask = find_contours(mask, 0.5)
        binary1 = (image < threshOtsu)
        #step 2: removing very thin line with binary opening
        binary1 =opening(binary1, disk(2))
        #step 3:just keep the most circular region, i.e. the lesion
        binary1 = findCircular(binary1)
        #step 4:fill in the holes of the segmentation
        binary1 = binary_fill_holes(binary1)

        #plot results
        fig = plt.figure(figsize=(17, 7))
        plt.figure(figsize=(20, 9))
        if(i==0):
            plt.suptitle(title)
        
        
        plt.subplot(2,2,1)
        plt.imshow(image, cmap="gray")
        for contour in contourMask:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        plt.subplot(2,2,2)
        plt.title(f"dice distance ={dice(binary1.ravel(),mask.ravel())}")
        plt.imshow(ComparVisu(image,binary1*np.max(image)), cmap="gray")
        resultSkinOtsu.append(binary1)
        for contour in contourMask:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        plt.show()


OtsuThreshSkin(skinImages, manualSkin,'skin',  'Otsu thresholding for skin lesions (custom)')

RESULTS OF THE CUSTOM OTSU-THRESHOLDING TECHNIQUE FOR SKIN LESIONS:
For each of the skin images, the dice distances are either lower (first 3 pictures) or almost the same (fourth picture). The main difference is the erasure of most hairs from the result.
This is due to the binary opening step that remove small edges (it worsened slightly the result for the fourth picture because it removed some thin details at the edges of the otsu segmentation. However, skin lesions are usually uniform smooth shapes). 
Keeping only the most circular component (not too small to not keep noise) helped only considering the skin lesion after the image faced binary operation.

This technique is more reliable than Otsu thesholding for skin lesions, given that otsu thresholding does not output a lesion shape with too many thin non-hair structures peaking out of it

In [ ]:
from sklearn.cluster import KMeans
#Note: the following code is for simple visual inspection. No dice/segmentation was done yet


#function to reorder the labels found from the k-mean algorithm from region with lowest pixel average to the highest 
#(to ensure consistency of cluster colors between the images)
def reorderLabel(image, labels):
    averages=np.zeros(np.max(labels)+1)
    for l in np.unique(labels):
        averages[l]= np.sum(image[labels==l])/np.sum(labels==l) #calculating connected component average intensity
    #mapping intensities to the correct connected components (0 for lowest average, 1 for higher, ...)
    sorted_indices=np.argsort(averages)
    mapping={old:new for new, old in enumerate(sorted_indices)}
    relabeled=np.vectorize(mapping.get)(labels)
    return relabeled

        
#function to do 4 k-means operations for each image (2,3,4 and 5 clusters respectively)    
def kmean(images, masks, part, title):
    #initialise clusters
    kmean2=KMeans(n_clusters=2)
    kmean3=KMeans(n_clusters=3)
    kmean4=KMeans(n_clusters=4)
    kmean5=KMeans(n_clusters=5)
    #iterate over images
    for i in range (0,4):
        image=images[i]
        #fitting k-means for each image
        kmean2.fit(image.reshape(-1, 1))
        kmean3.fit(image.reshape(-1, 1))
        kmean4.fit(image.reshape(-1, 1))
        kmean5.fit(image.reshape(-1, 1))
        if(part=='abdo'):
            maskK=masks[0][i]
            maskT=masks[1][i]
            maskK_c= find_contours(masks[0][i], 0.5)
            maskT_c= find_contours(masks[1][i], 0.5)
        else:
            mask=masks[i]
            contourMask = find_contours(mask, 0.5)
            
        #visualizing clusteredresults
        fig = plt.figure(figsize=(17, 7))
        plt.figure(figsize=(20, 9))
        if(i==0):
            plt.suptitle(title)
        
        if part=='abdo':
            plt.subplot(2,5,1)
            plt.imshow(image, cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
            plt.subplot(2,5,2)
            plt.imshow(reorderLabel(image,kmean2.labels_.reshape(image.shape)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
            plt.subplot(2,5,3)
            plt.imshow(reorderLabel(image,kmean3.labels_.reshape(image.shape)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
            plt.subplot(2,5,4)
            plt.imshow(reorderLabel(image,kmean4.labels_.reshape(image.shape)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
            plt.subplot(2,5,5)
            plt.imshow(reorderLabel(image,kmean5.labels_.reshape(image.shape)), cmap="gray")
            for contour in maskK_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            for contour in maskT_c:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='b')
            plt.show()
            
        else:
            plt.subplot(2,5,1)
            plt.imshow(image, cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.subplot(2,5,2)
            plt.imshow(reorderLabel(image,kmean2.labels_.reshape(image.shape)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.subplot(2,5,3)
            plt.imshow(reorderLabel(image,kmean3.labels_.reshape(image.shape)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.subplot(2,5,4)
            plt.imshow(reorderLabel(image,kmean4.labels_.reshape(image.shape)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.subplot(2,5,5)
            plt.imshow(reorderLabel(image,kmean5.labels_.reshape(image.shape)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.show()

#clustering for all modalities (4 images per modality)
kmean(skinImages, manualSkin, 'skin','k-means for skin lesions (k = 2, 3, 4, 5 left to right)')
kmean(abdoImages, [manualAbdoKidney,manualAbdoTumor],'abdo','k-means for abdominal CT (k = 2, 3, 4, 5 left to right)')
kmean(brainImages,manualBrain,'brain', 'k-means for brain mri (k = 2, 3, 4, 5 left to right)')
kmean(heartImages,manualHeart, 'heart','k-means for heart mri (k = 2, 3, 4, 5 left to right)')


CONCLUSIONS FOR KMEANS:
- For skin images, dark clusters encapsulate most of the skin lesion (due to the high dark contrast with the rest of the image), they also encapsulate the hair which need to be removed for segmentation. When we increase the number of clusers, we find the non-white regions in the center increase in the volume, which would make the segmentation more accurate. An interesting strategy would be to choose a clustering in between 0 and the highest cluster to not keep the hair while keeping most of the skin lesion.
- For abdominal CTs, when we choose 3-4 clusters, kidneys are highlited in white but some operations based on shape are needed to identify them
- The results for Brain MRIs remain inconsistent, needing to choose different numbers of clusters between images to identify the corpus callosum
- For heart MRI, the left and right ventricules become visible when we choose 2,4 or 5 clusters. 4 clusters is the safest choice to keep the most volume of the left ventricule (in comparison with 5 clusters) and having this structure the least attached possible from the right ventricule (in comparison with 2 clusters) that share the same label. 

In [ ]:
#custom method based on k-means for skin lesions
from skimage.morphology import remove_small_objects

def reorderLabel(image, labels):
    averages=np.zeros(np.max(labels)+1)
    for l in np.unique(labels):
        averages[l]= np.sum(image[labels==l])/np.sum(labels==l)
    sorted_indices=np.argsort(averages)
    mapping={old:new for new, old in enumerate(sorted_indices)}
    relabeled=np.vectorize(mapping.get)(labels)
    return relabeled

backgroundSkinPts=[]        
    
def SkinKmean(images, masks, part, title):
    #k-mean with 4 clusters (to retain the biggest lesion boundary possible)
    kmean4=KMeans(n_clusters=4)
    #iterate over images
    for i in range (0,4):
        image=images[i]
        #fit to image
        kmean4.fit(image.reshape(-1, 1))
        
        mask=masks[i]
        contourMask = find_contours(mask, 0.5)
        
        fig = plt.figure(figsize=(17, 7))
        plt.figure(figsize=(20, 9))
        if(i==0):
            plt.suptitle(title)
        
        plt.subplot(2,5,1)
        plt.imshow(image, cmap="gray")
        for contour in contourMask:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        plt.subplot(2,5,2)
        #reorder labels
        clustered=reorderLabel(image,kmean4.labels_.reshape(image.shape))
        
        #save bright clusters as background regions for later
        backgroundSkinPts.append(remove_small_objects((clustered>2), min_size=100)) 
        
        #keep the 2 darkest clusters
        clustered= clustered<2
        #opening to remove thin hairs 
        clustered =opening(clustered, disk(2))
        #find the most circular component (i.e. lesion)
        clustered = findCircular(clustered)
        #fill the holes of the segmentation
        clustered = binary_fill_holes(clustered)
        plt.imshow(ComparVisu(clustered,image), cmap="gray")
        for contour in contourMask:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.title(f"dice distance ={dice(clustered.ravel(),mask.ravel())}")
        
        plt.show()

SkinKmean(skinImages, manualSkin, 'skin','k-means for skin lesions (custom)')

RESULTS (CUSTOM K-MEANS METHOD FOR SKIN LESIONS):
This technique present the same advantages as the ones cited above for the custom otsu thresholding method. It performs slightly worse, a result highly sensitive to the numbers of clusters chosen at first.

In [ ]:
#custom method based on k-means for the heart left ventricule segmentation 
#(very similar to the custom method for skin lesions)
from skimage.morphology import remove_small_objects



def reorderLabel(image, labels):
    averages=np.zeros(np.max(labels)+1)
    for l in np.unique(labels):
        averages[l]= np.sum(image[labels==l])/np.sum(labels==l)
    sorted_indices=np.argsort(averages)
    mapping={old:new for new, old in enumerate(sorted_indices)}
    relabeled=np.vectorize(mapping.get)(labels)
    return relabeled

backgroundHeartPts=[]        
    
def HeartKmean(images, masks, part, title):
    kmean4=KMeans(n_clusters=4) 
    for i in range (0,4):
        image=images[i]
        kmean4.fit(image.reshape(-1, 1))
        
        mask=masks[i]
        contourMask = find_contours(mask, 0.5)
        
        fig = plt.figure(figsize=(17, 7))
        plt.figure(figsize=(20, 9))
        if(i==0):
            plt.suptitle(title)
        
        plt.subplot(2,5,1)
        plt.imshow(image, cmap="gray")
        for contour in contourMask:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        plt.subplot(2,5,2)
        clustered=reorderLabel(image,kmean4.labels_.reshape(image.shape))
        backgroundHeartPts.append(remove_small_objects((clustered>2), min_size=100))
        clustered= clustered==2
        #optimised disk size for ventricule segmentation
        clustered =opening(clustered, disk(1))
        #circularity parameters optimised for ventricule segmentation
        clustered = findCircular(clustered,0.008,0.9) 
        clustered = binary_fill_holes(clustered)
        plt.imshow(ComparVisu(image,clustered*np.max(image)), cmap="gray")
        for contour in contourMask:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.title(f"dice distance ={dice(clustered.ravel(),mask.ravel())}")
        
        plt.show()

HeartKmean(heartImages, manualHeart, 'heart','k-means for heart ventricule (custom)')

RESULTS (CUSTOM K-MEANS METHOD FOR SKIN LESIONS): 
As explained above, 4 clusters were chosen for the following reason:  the left and right ventricules become visible when we choose 2,4 or 5 clusters. 4 clusters is the safest choice to keep the most volume of the left ventricule (in comparison with 5 clusters) and having this structure the least attached possible from the right ventricule (in comparison with 2 clusters) that share the same label.

The binary opening step was done to remove remaining thin connection between the left and right ventricule, for these two ventricules to not be considered as a single connected component.

The findCircular function is used to find the most circular component again (the left ventricule)

The results found were mostly encapsulated inside the original manual mask and do not exceed a dice distance of 0.15, making this technique somewhat reliable

*2*. Initialize a segmentation by region growing, watersheds or deformable models from the previous results (the method can vary according to the images and the subsequent segmentation method), and try to improve this initialization using pre- and post-processing (filtering, etc.).



In [ ]:
#algorithm that finds seed regions for skin images
print("Seed points chosen for skin images")

#saves a circular region in the middle of the found custom otsu segmentation
#(after applying a big opening operation to remove branching uneccessary details)
regionsSkin=[] 

#saves the  found custom otsu segmentation as a seed region 
#(after applying a big opening operation to remove branching uneccessary details)
regionsSkinFull=[]
centersSkin=[]
for i in range (0,4):
    #removes branching unecessary details that might extend out of the manual mask region
    opened=opening(resultSkinOtsu[i], disk(19))
    regionsSkinFull.append(opened)
    #calculate the center of the segmentation region
    coords=np.argwhere(opened==1)
    center=coords.mean(axis=0)
    centersSkin.append(center)
    #only keep points that are in the predicted segmentation and at half "radius" distance of the segmented region 
    #(to not risk have details outside the correct manual mask)
    distances=np.sqrt((coords[:,0]-center[0])**2 + (coords[:,1]-center[1])**2)
    maxDistance=distances.max()
    maskCircle=np.zeros_like(opened)
    maskCircle[coords[distances<= 0.5*maxDistance][:,0], coords[distances<= 0.5*maxDistance][:,1]]=1
    regionsSkin.append(maskCircle)
    
for i in range (0,4):
    plt.imshow(ComparVisu(skinImages[i],regionsSkin[i]),cmap="grey")
    contourSkin = find_contours(manualSkin[i], 0.5)
    for contour in contourSkin:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
    plt.show()



3. Segmentation by region growing




In [ ]:
from scipy.ndimage import distance_transform_edt

#expands the initial region slightly (for a single point)
def initialize_segmentation(seed_pt,img_shape):
    seg_init = np.zeros(img_shape).astype(float)
    seg_init[ seed_pt[0], seed_pt[1]] = 1
    dist = distance_transform_edt(np.abs(1-seg_init))

    # initialise the seeded region with a certain radius
    size_radius = 5
    seg_init = dist < size_radius
    return seg_init

#expands the initial region slightly (for an initial region)
def initialize_segmentation_region(seed_region,img_shape):
    seg_init = np.zeros(img_shape).astype(float)
    seg_init[seed_region==1] = 1
    dist = distance_transform_edt(np.abs(1-seg_init))

    # initialise the seeded region with a certain radius
    size_radius = 5
    seg_init = dist < size_radius
    return seg_init


#region growing (for a single point)
def segmentation_region_growing(img,seed_pt,tau,struct):

    # define the neighbourhood (for instance 4-connected pixels)
    se=strel('diamond',struct)

    seg_init = initialize_segmentation(seed_pt,img.shape)
    seg_n_plus_1 = seg_init
    seg_n = np.zeros(seg_n_plus_1.shape)

    # loop while the region can still keep growing
    while( np.abs(seg_n_plus_1 != seg_n).sum() != 0):
        seg_n = seg_n_plus_1
        seg_n_plus_1 = morpho.dilation(seg_n, se).astype(seg_n.dtype)
        # calculate average value
        avg = np.sum( img[seg_n>0] ) / ( float(seg_n.sum()))
        seg_n_plus_1 = np.logical_and( seg_n_plus_1 , np.abs( img-avg ) < tau)
        # do not lose previous points of the segmentation
        seg_n_plus_1 = np.logical_or( seg_n_plus_1 , seg_n)

    return seg_n,seg_init

#region growing (for an initial region)
def segmentation_region_growing_region(img,seed_pt,tau, struct):

    # define the neighbourhood (for instance 4-connected pixels)
    se=strel('diamond',struct)

    seg_init = initialize_segmentation_region(seed_pt,img.shape)
    seg_n_plus_1 = seg_init
    seg_n = np.zeros(seg_n_plus_1.shape)

    # loop while the region can still keep growing
    while( np.abs(seg_n_plus_1 != seg_n).sum() != 0):
        seg_n = seg_n_plus_1
        seg_n_plus_1 = morpho.dilation(seg_n, se).astype(seg_n.dtype)
        # calculate average value
        avg = np.sum( img[seg_n>0] ) / ( float(seg_n.sum()))
        seg_n_plus_1 = np.logical_and( seg_n_plus_1 , np.abs( img-avg ) < tau)
        # do not lose previous points of the segmentation
        seg_n_plus_1 = np.logical_or( seg_n_plus_1 , seg_n)

    return seg_n,seg_init

im=imG

#useful to find point coordinates and gray values:
#%matplotlib notebook
#plt.imshow(imG,cmap='gray')


   
#structring size=1
for i in range (0,4):
    #points
    seed_pt = np.asarray([110,130])   # initialization with only one point - Try to modify the code to initialize with one region obtained previously
    for j in range (1,6):
        tau = j*0.02 #for float images, otherwise choose an integer value
        img_out_region_growing,seg_init = segmentation_region_growing(skinImages[i],seed_pt,tau,1)
        if j==1:
            fig = plt.figure(figsize=(22, 22))
            grid = AxesGrid(fig, 111,
                            nrows_ncols = (1, 7),
                            axes_pad = 0.5)
            grid[0].imshow(skinImages[i],cmap='gray')
            grid[0].axis('off')
            grid[0].set_title("image")
            grid[1].imshow(seg_init,cmap='gray')
            grid[1].axis('off')
            grid[1].set_title("Initial region")
        contourSkin = find_contours(manualSkin[i], 0.5)
        for contour in contourSkin:
                grid[j+1].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        grid[j+1].imshow(ComparVisu(skinImages[i],img_out_region_growing),cmap='gray')
        grid[j+1].axis('off')
        grid[j+1].set_title(f"size=1, tau={tau},dist={round(dice(img_out_region_growing.ravel(),manualSkin[i].ravel()),3)}")
    
    
    #regions
    seed_region = regionsSkin[i]  # initialization with only one point - Try to modify the code to initialize with one region obtained previously
    for j in range (1,6):
        tau = j*0.02 #for float images, otherwise choose an integer value
        img_out_region_growing,seg_init = segmentation_region_growing_region(skinImages[i],seed_region,tau,1)
        if j==1:
            fig = plt.figure(figsize=(22, 22))
            grid = AxesGrid(fig, 111,
                            nrows_ncols = (1, 7),
                            axes_pad = 0.5)
            grid[0].imshow(skinImages[i],cmap='gray')
            grid[0].axis('off')
            grid[0].set_title("image")
            grid[1].imshow(seg_init,cmap='gray')
            grid[1].axis('off')
            grid[1].set_title("Initial region")
        for contour in contourSkin:
                grid[j+1].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        grid[j+1].imshow(ComparVisu(skinImages[i],img_out_region_growing),cmap='gray')
        grid[j+1].axis('off')
        grid[j+1].set_title(f"size=1, tau={tau},dist={round(dice(img_out_region_growing.ravel(),manualSkin[i].ravel()),3)}")

        
#structring size=3
for i in range (0,4):
    #points
    seed_pt = np.asarray([110,130])   # initialization with only one point - Try to modify the code to initialize with one region obtained previously
    for j in range (1,6):
        tau = j*0.02 #for float images, otherwise choose an integer value
        img_out_region_growing,seg_init = segmentation_region_growing(skinImages[i],seed_pt,tau,3)
        if j==1:
            fig = plt.figure(figsize=(22, 22))
            grid = AxesGrid(fig, 111,
                            nrows_ncols = (1, 7),
                            axes_pad = 0.5)
            grid[0].imshow(skinImages[i],cmap='gray')
            grid[0].axis('off')
            grid[0].set_title("image")
            grid[1].imshow(seg_init,cmap='gray')
            grid[1].axis('off')
            grid[1].set_title("Initial region")
        contourSkin = find_contours(manualSkin[i], 0.5)
        for contour in contourSkin:
                grid[j+1].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        grid[j+1].imshow(ComparVisu(skinImages[i],img_out_region_growing),cmap='gray')
        grid[j+1].axis('off')
        grid[j+1].set_title(f"size=3, tau={tau},dist={round(dice(img_out_region_growing.ravel(),manualSkin[i].ravel()),3)}")
    
    
    #regions
    seed_region = regionsSkin[i]  # initialization with only one point - Try to modify the code to initialize with one region obtained previously
    for j in range (1,6):
        tau = j*0.02 #for float images, otherwise choose an integer value
        img_out_region_growing,seg_init = segmentation_region_growing_region(skinImages[i],seed_region,tau,3)
        if j==1:
            fig = plt.figure(figsize=(22, 22))
            grid = AxesGrid(fig, 111,
                            nrows_ncols = (1, 7),
                            axes_pad = 0.5)
            grid[0].imshow(skinImages[i],cmap='gray')
            grid[0].axis('off')
            grid[0].set_title("image")
            grid[1].imshow(seg_init,cmap='gray')
            grid[1].axis('off')
            grid[1].set_title("Initial region")
        for contour in contourSkin:
                grid[j+1].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        grid[j+1].imshow(ComparVisu(skinImages[i],img_out_region_growing),cmap='gray')
        grid[j+1].axis('off')
        grid[j+1].set_title(f"size=3, tau={tau},dist={round(dice(img_out_region_growing.ravel(),manualSkin[i].ravel()),3)}")



CONCLUSION ON REGION GROWING:
We have tested the segmentation by region growing for 4 skin images while changing several parameters.

-The initial region is crucial. When we choose a random single point, it can only expend around that point (therefore choosing an initial mask inside the expected boundaries is crucial). And the growth can be limited since the initial mask is small.
-Results depend heavily on the "tau" variable that represent the intensity difference allowed to grow the region. A higher tau means a bigger final region
-Results depend also heavily on the structuring size which decides how much the region can grow at each iteration. If the structuring size is too big, it will grow more but the result will be more noisy (since the algorithm works on a lower resolution/precision level)


Note: the intial region chosen is the circle inside the segmentation of the otsu custom thresholding method (explain above)

4. Segmentation by watersheds with *markers*

In [ ]:
#disk size=1
for ima in range (0,4):
    fig = plt.figure(figsize=(17, 7))
    grid = AxesGrid(fig, 111,
                    nrows_ncols = (1, 5),
                    axes_pad = 0.5)
    
    

    im=skinImages[ima]*255
    
    grid[0].imshow(im, cmap='gray')
    grid[0].axis('off')
    grid[0].set_title("Original image")
    #morphological gradient
    se=morpho.disk(1)
    grad=morpho.dilation(im,se)-morpho.erosion(im,se)

    # Closing of the gradient
    grad = morpho.closing(grad, morpho.disk(1))
    plt.imshow(grad, cmap="gray")
    grid[1].imshow(grad,cmap='gray')
    grid[1].axis('off')
    grid[1].set_title("morphological gradient (disk size=1)")

    init = regionsSkin[ima]*255
    
        
    init[backgroundSkinPts[ima]==1]=255
    

    markers = ndi.label(init)[0]
    grid[2].imshow(markers,cmap='gray')
    grid[2].axis('off')
    grid[2].set_title("init markers")

    labels = watershed(grad, markers,watershed_line=True)
    grid[3].imshow(labels,cmap='gray')
    grid[3].axis('off')
    grid[3].set_title("labels")

    labeltoChoose=labels[int(centersSkin[ima][0])][int(centersSkin[ima][1])]
    segmented= (labels==labeltoChoose)*1
    grid[4].imshow(ComparVisu(im,segmented*255),cmap='gray')
    grid[4].axis('off')
    contourSkin = find_contours(manualSkin[ima], 0.5)
    for contour in contourSkin:
        grid[4].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')

    grid[4].set_title(f"segmentation, dist={round(dice(segmented.ravel(),manualSkin[ima].ravel()),3)}")
    


#disk size=3
for ima in range (0,4):
    fig = plt.figure(figsize=(17, 7))
    grid = AxesGrid(fig, 111,
                    nrows_ncols = (1, 5),
                    axes_pad = 0.5)
    
    

    im=skinImages[ima]*255
    
    grid[0].imshow(im, cmap='gray')
    grid[0].axis('off')
    grid[0].set_title("Original image")
    #morphological gradient
    se=morpho.disk(3)
    grad=morpho.dilation(im,se)-morpho.erosion(im,se)

    # Closing of the gradient
    grad = morpho.closing(grad, morpho.disk(3))
    plt.imshow(grad, cmap="gray")
    grid[1].imshow(grad,cmap='gray')
    grid[1].axis('off')
    grid[1].set_title("morphological gradient  (disk size=3)")

    init = regionsSkin[ima]*255
    
        
    init[backgroundSkinPts[ima]==1]=255
    

    markers = ndi.label(init)[0]
    grid[2].imshow(markers,cmap='gray')
    grid[2].axis('off')
    grid[2].set_title("init markers")

    labels = watershed(grad, markers,watershed_line=True)
    grid[3].imshow(labels,cmap='gray')
    grid[3].axis('off')
    grid[3].set_title("labels")

    labeltoChoose=labels[int(centersSkin[ima][0])][int(centersSkin[ima][1])]
    segmented= (labels==labeltoChoose)*1
    grid[4].imshow(ComparVisu(im,segmented*255),cmap='gray')
    grid[4].axis('off')
    contourSkin = find_contours(manualSkin[ima], 0.5)
    for contour in contourSkin:
        grid[4].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')

    grid[4].set_title(f"segmentation, dist={round(dice(segmented.ravel(),manualSkin[ima].ravel()),3)}")
    

CONCLUSION ON WATERSHED WITH MARKERS:
We have tested the segmentation by region growing for 4 skin images on 2 different gradient disk size.

-The initial regions are crucial. We need to choose an initial region inside the expected manual mask so that it can start to grow from a correct region corresponding to a part of the skin lesion. The background regions should also set representatively and not too close (to not enter the lesion region) and not too far (to limit the expansion of the lesion region) from the lesion boundaries

-Changing the structuring disk size does not influence the final results much since because watershed growing depend heavily on edges positions rather than edges thickness.



Note: the intial region chosen is the circle inside the segmentation of the otsu custom thresholding method (explain above)
And the initial background regions are the ones found above on the k-means custom method (explain above)

5. Segmentation using deformable models
- parametric model, contour-based segmentation
- implicit model, using level sets (Chan & Vese), region-based segmentation

In [ ]:
# Segmentation by active contours 
from skimage.draw import polygon


def setGrid(im, a,b, w_e, w_l, g, grid, index,titleStart, init):
    snake = active_contour(gaussian(im, 0.1),
                           init[0], alpha=a, beta=b, w_edge=w_e, w_line=w_l, gamma=g)
    final=np.zeros_like(im)
    poly=polygon(snake[:, 0], snake[:, 1], final.shape)
    final[poly[0], poly[1]]=1
    grid[index].imshow(ComparVisu(im,final*255),cmap='gray')
    grid[index].axis('off')
    contourSkin = find_contours(manualSkin[ima], 0.5)
    for contour in contourSkin:
        grid[index].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
    grid[index].set_title(f"{titleStart}, dist={round(dice(final.ravel(),manualSkin[ima].ravel()),3)}")
    return grid

for ima in range (0,4):
    #arbitrary circle
    fig = plt.figure(figsize=(17, 7))
    grid = AxesGrid(fig, 111,
                    nrows_ncols = (1, 3),
                    axes_pad = 0.5)
    
    

    im=skinImages[ima]*255
    grid[0].imshow(im, cmap='gray')
    grid[0].axis('off')
    grid[0].set_title("Original image")

    s = np.linspace(0, 2*np.pi, 100)
    r = 110 + 30*np.sin(s)
    c = 130 + 20*np.cos(s)
    init = np.array([r, c]).T
    
    grid[1].imshow(im, cmap='gray')
    grid[1].axis('off')
    grid[1].set_title("Initial region")
    grid[1].plot(init[:,1], init[:,0], linewidth=2, c='b')

    snake = active_contour(gaussian(im, 0.1),
                           init, alpha=0.1, beta=10, w_edge=100, w_line=100, gamma=0.01)

    final=np.zeros_like(im)
    poly=polygon(snake[:, 0], snake[:, 1], final.shape)
    final[poly[0], poly[1]]=1

    
    grid[2].imshow(ComparVisu(im,final*255),cmap='gray')
    grid[2].axis('off')
    contourSkin = find_contours(manualSkin[ima], 0.5)

    for contour in contourSkin:
        grid[2].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')

    grid[2].set_title(f"segmentation, dist={round(dice(final.ravel(),manualSkin[ima].ravel()),3)}")


    plt.show()
    
    
    #custom region
    fig = plt.figure(figsize=(17, 7))
    grid = AxesGrid(fig, 111,
                    nrows_ncols = (1, 3),
                    axes_pad = 0.5)
    
    

    im=skinImages[ima]*255
    grid[0].imshow(im, cmap='gray')
    grid[0].axis('off')
    grid[0].set_title("Original image")

    
    init = find_contours(regionsSkinFull[ima], 0.5)
    
    grid[1].imshow(im, cmap='gray')
    grid[1].axis('off')
    grid[1].set_title("Initial region")
    grid[1].plot(init[0][:,1], init[0][:,0], linewidth=2, c='b')

    snake = active_contour(gaussian(im, 0.1),
                           init[0], alpha=0.1, beta=10, w_edge=100, w_line=100, gamma=0.01)

    final=np.zeros_like(im)
    poly=polygon(snake[:, 0], snake[:, 1], final.shape)
    final[poly[0], poly[1]]=1

    
    grid[2].imshow(ComparVisu(im,final*255),cmap='gray')
    grid[2].axis('off')
    contourSkin = find_contours(manualSkin[ima], 0.5)

    for contour in contourSkin:
        grid[2].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')

    grid[2].set_title(f"segmentation, dist={round(dice(final.ravel(),manualSkin[ima].ravel()),3)}")

    plt.show()
    
    #experimenting with the parameters
    fig = plt.figure(figsize=(17, 7))
    grid = AxesGrid(fig, 111,
                    nrows_ncols = (1, 5),
                    axes_pad = 0.5)
    grid=setGrid(im, 0.8, 10, 100, 100, 0.01, grid, 0, "(increased alpha)",init)
    grid=setGrid(im, 0.1, 30, 100, 100, 0.01, grid, 1, "(increased beta)",init)
    grid=setGrid(im, 0.1, 10, 300, 100, 0.01, grid, 2, "(increased w_edge)",init)
    grid=setGrid(im, 0.1, 10, 100, 300, 0.01, grid, 3, "(increased w_line)",init)
    grid=setGrid(im, 0.1, 10, 100, 100, 0.1, grid, 4, "(increased gamma)",init)

    
    plt.show()



CONCLUSIONS ON ACTIVE CONTOURS:

(We have tested the active contours on skin images while changing several parameters: alpha, beta, w_edge, w_line and gamma)

-The initial region chosen is crucial again, for the same reasons as the segmentation methods above
-The technique is very sensitive to the parameters we give it, for example increasing w_line increases attraction towards brighter regions (especially seen on the second set of pictures) therefore result in segmentations that go outside the boundaries of the expected mask

In [ ]:
#Segmentation using level sets and region information
for ima in range (0,4):
    #disk in the middle of the image
    fig = plt.figure(figsize=(17, 7))
    grid = AxesGrid(fig, 111,
                    nrows_ncols = (1, 4),
                    axes_pad = 0.5)
    
    
    image = skinImages[ima]

    # Init with checkerboard
    #init_ls = checkerboard_level_set(image.shape, 6)

    # Init with disks
    init_ls = disk_level_set (image.shape)

    # Init avec plusieurs cercles
    #circleNum = 8
    #circleRadius = image.shape[0] / (3*circleNum)
    #circleStep0 = image.shape[0]/(circleNum+1)
    #circleStep1 = image.shape[1]/(circleNum+1)
    #init_ls = np.zeros(image.shape)
    #for i in range(circleNum):
    #        for j in range(circleNum):
    #            init_ls = init_ls + circle_level_set (image.shape, 
    #                                                  ((i+1)*circleStep0, (j+1)*circleStep1), circleRadius)


    cv = chan_vese(image, mu=0.25, lambda1=5, lambda2=1, tol=1e-3, max_num_iter=200,
                   dt=0.5, init_level_set=init_ls, extended_output=True)

    
    grid[0].imshow(image, cmap='gray')
    grid[0].axis('off')
    grid[0].set_title("Original image")
    
    
    grid[1].imshow(ComparVisu(image,init_ls), cmap='gray')
    grid[1].axis('off')
    grid[1].set_title("initial level set")
    
    grid[2].imshow(ComparVisu(image,cv[0]), cmap='gray')
    grid[2].axis('off')
    contourSkin = find_contours(manualSkin[ima], 0.5)
    for contour in contourSkin:
        grid[2].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
    grid[2].set_title(f"Segmentation - {len(cv[2])} iters, dist={round(dice(cv[0].ravel(),manualSkin[ima].ravel()),3)}")


    grid[3].imshow(cv[1], cmap="gray")
    grid[3].axis('off')
    grid[3].set_title("Final Level Set", fontsize=12)
    plt.show()
    plt.figure(figsize=(8, 4))

    plt.plot(cv[2])
    plt.title("Evolution of energy over iterations", fontsize=12)

    plt.show()
    
    
    
    
    
    
    #result from previous segmentation
    fig = plt.figure(figsize=(17, 7))
    grid = AxesGrid(fig, 111,
                    nrows_ncols = (1, 4),
                    axes_pad = 0.5)
    
    # Init with previous segmentation
    init_ls = regionsSkinFull[ima]

    cv = chan_vese(image, mu=0.25, lambda1=5, lambda2=1, tol=1e-3, max_num_iter=200,
                   dt=0.5, init_level_set=init_ls, extended_output=True)

    
    grid[0].imshow(image, cmap='gray')
    grid[0].axis('off')
    grid[0].set_title("Original image")
    
    
    grid[1].imshow(ComparVisu(image,init_ls), cmap='gray')
    grid[1].axis('off')
    grid[1].set_title("initial level set")
    
    grid[2].imshow(ComparVisu(image,cv[0]), cmap='gray')
    grid[2].axis('off')
    contourSkin = find_contours(manualSkin[ima], 0.5)
    for contour in contourSkin:
        grid[2].plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
    grid[2].set_title(f"Segmentation - {len(cv[2])} iters, dist={round(dice(cv[0].ravel(),manualSkin[ima].ravel()),3)}")


    grid[3].imshow(cv[1], cmap="gray")
    grid[3].axis('off')
    grid[3].set_title("Final Level Set", fontsize=12)
    plt.show()
    plt.figure(figsize=(8, 4))

    plt.plot(cv[2])
    plt.title("Evolution of energy over iterations", fontsize=12)

    plt.show()

6. Segmentation of an image sequence - On the cardiac MRI images, propose a segmentation method that uses the result in a frame to initialize the result in the next frame (use for instance slice z=2 or z-3). Illustrate and discuss the results. Is it relevant to use a similar approach to use the segmentation of a slice to guide the segmentation of the next slide? Discuss.

In [ ]:
#custom heart method along z-axis (based on k-means)
from skimage.morphology import remove_small_objects

       
    
def HeartKmeanAlongZ(images, masks, part, title):
    #step 1, for the first slice we want to analyze (i.e. slice 3 in our case) apply same technique as above 
    #custom k-means clustering
    kmean4=KMeans(n_clusters=4)
    for i in range (0,4):
        image=heartImages[i]
        kmean4.fit(image.reshape(-1, 1))
        
        mask=masks[i]
        contourMask = find_contours(mask, 0.5)
        
        fig = plt.figure(figsize=(17, 7))
        plt.figure(figsize=(20, 9))
        if(i==0):
            plt.suptitle(title)
        
        plt.subplot(2,5,1)
        plt.title("z=3 (first slice analyzed)")
        plt.imshow(image, cmap="gray")
        for contour in contourMask:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
        plt.subplot(2,5,2)
        clustered=reorderLabel(image,kmean4.labels_.reshape(image.shape))
        clustered= clustered==2
        clustered =opening(clustered, disk(1))
        clustered = findCircular(clustered,0.008,0.9)
        clustered = binary_fill_holes(clustered)
        plt.imshow(ComparVisu(image,clustered*np.max(image)), cmap="gray")
        for contour in contourMask:
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.title(f"dice distance ={dice(clustered.ravel(),mask.ravel())}")
        
        plt.show()
        
        for j in range (4,7):
            
            fig = plt.figure(figsize=(17, 7))
            plt.figure(figsize=(20, 9))
            
            #step 2: save region of interest for subsequent slices along the z-axis
            #the ROI correspond to a circle of radius 1.2 times the maximum distance centroid-point in the last segmentation
            images=data
            image=images[:,:,j,i]
            coords=np.argwhere(clustered==1)
            center=coords.mean(axis=0)
            distances=np.sqrt((coords[:,0]-center[0])**2 + (coords[:,1]-center[1])**2)
            maxDistance=distances.max()
            ROI=image.copy()
            indexes=np.indices(image.shape)
            dist_map=np.sqrt((indexes[0]-center[0])**2+(indexes[1]-center[1])**2)
            ROI[dist_map>1.2*maxDistance]=0
            
            maskPts=seg[j,i]
            zeroIndex=np.where((maskPts[:,0]==0)&(maskPts[:,1]==0))[0][0]
            maskPts=maskPts[0:zeroIndex]
            splineInfo,numPts=splprep([maskPts[:,1],maskPts[:,0]])
            morePts=np.linspace(0, 1, 100)
            ptsUpdated=splev(morePts, splineInfo)
            mask=np.zeros_like(imHeart)
            poly=polygon(ptsUpdated[0], ptsUpdated[1], mask.shape)
            mask[poly[0],poly[1]]=1
            
            #step 3: use same technique to segment the next slice 
            #we take 3 clusters this time since we work with a smaller region. 
            #If we chose 4, we would lose to much information on the left ventricule
            kmean5=KMeans(n_clusters=3)
            kmean5.fit(ROI.reshape(-1, 1))
            clustered=reorderLabel(ROI,kmean5.labels_.reshape(ROI.shape))
            plt.subplot(2,6,(j-4)*2+1)
            plt.title(f"z={j}, clustered ROI")
            plt.imshow(clustered, cmap="gray")
            clustered= clustered==2
            clustered =opening(clustered, disk(1))
            clustered = findCircular(clustered,0.008,0.9)
            clustered = binary_fill_holes(clustered)
            plt.subplot(2,6,(j-4)*2+2)
            plt.imshow(ComparVisu(image,clustered*np.max(image)), cmap="gray")
            for contour in contourMask:
                plt.plot(contour[:, 1], contour[:, 0], linewidth=2, c='r')
            plt.title(f"dice distance ={dice(clustered.ravel(),mask.ravel())}")
            if(j==6):
                plt.show()
            


HeartKmeanAlongZ(heartImages, manualHeart, 'heart','k-means for heart ventricule (custom)')

CONCLUSIONS ON THE CUSTOM HEART SEGMENTATION METHOD ALONG THE Z-AXIS:
-With a dice-distance that do not exceed 0.21, this method is semi-reliable. 
-It results from the assumption that the centroid of the left ventricule does not move much from a slice to the other in the z-axis
-Its main weakness is the loss of information that happens when we cluster the region in the new ROI (since a much smaller region, that has the left ventricule take up most the area, needs to be segmented into multiple regions)
-Its main advantage is ensuring that the segmentation doesn't move much farther from the expected manual mask segmentation curve

Another technique that could have been implemented is registration, which aligns the slices so that the left ventricle overlap as closely as possible by transforming each slice using rigid/affine transformations. The segmentation from one slice would be propagated more accurately to the next but relies heavily on an accurate initial segmentation.